In [26]:
import pandas as pd
from PyPDF2 import PdfReader as PDFReader, PdfWriter as PDFWriter
import os

In [27]:
os.listdir('./')

['.ipynb_checkpoints', 'BRGY TUGATOG.pdf', 'PDFS', 'test.ipynb']

In [109]:
params = {
    'number': {
        'upper_left': (0, 630),
        'lower_right': (60, 80),
        'upper_right': (60, 630),
        'lower_left': (0, 80),
    },
    'legend_and_voters_name': {
        'upper_left': (60, 630),
        'lower_right': (360, 80),
        'upper_right': (360, 630),
        'lower_left': (60, 80),
    },
    'address': {
        'upper_left': (360, 630),
        'lower_right': (580, 80),
        'upper_right': (580, 630),
        'lower_left': (360, 80),
    }
}

end_line = "We CERTIFY that the above voting records are true, correct and complete."

In [97]:
# checks if the current page contains the end of the list for a certain precinct number
cur = PDFReader('BRGY TUGATOG.pdf')
list_ends = []
for i in range(len(cur.pages)):
    lst = cur.pages[i].extract_text().split('\n')
    check_end_line = False
    for ind in range(len(lst)):
        if lst[ind] == end_line:
            check_end_line = True
            list_ends.append(i)
            break

In [129]:
# assumption here is that the end of the list page with the name and signature texts contains table entries
# if there are no table entries, we basically skip it and move on to the next precinct
cur = PDFReader('BRGY TUGATOG.pdf')
output = PDFWriter()
# print(cur.pages[15].extract_text())
page_content = cur.pages[15].extract_text().split('\n')
# print(page_content)
found = False
first_number = int(page_content[8].split()[0])
# print(first_number)
for i in range(len(page_content)):
    if page_content[i] == end_line:
        print("hala")
    if page_content[i] == 'Name and Signature' and not found:
        print(page_content[i-1].split()[0], i, "whjat")
        found = True
        for key, index in enumerate(params):
            pg = cur.pages[15]
            ind = params[index]
            diff = abs(first_number - int(page_content[i-1].split()[0]))+1
            pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
            pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
            pg.cropbox.lower_left = (ind['upper_left'][0], ind['upper_left'][1]-diff*20)
            pg.cropbox.lower_right = (ind['upper_right'][0], ind['upper_right'][1]-diff*20)
            output.add_page(pg)
with open(f'test.pdf', "wb") as out_f:
    output.write(out_f)

194 16 whjat
hala


In [141]:
cur = PDFReader(f'BRGY TUGATOG.pdf')
output = PDFWriter()
counter = 0
for i in range(len(cur.pages)):
    page_content = cur.pages[i].extract_text().split('\n')
    check_end, current_index, name_and_signature_found = False, -1, False
    for ind in range(len(page_content)):
        if page_content[ind] == "Name and Signature" and not name_and_signature_found:
            check_end, name_and_signature_found = True, True
            # handle case where the end of the list for a certain precinct is found
            first_number = page_content[8].split()[0]
            if first_number == 'Name':
                counter += 1
                # handle the case where the eight index value is not a valid table entry (Name and Signature)
                continue
            else:
                # else, find the last number of the last person for that precinct and proceed with cropping
                diff = abs(int(first_number) - int(page_content[ind-1].split()[0]))+1
                
                for key, index in enumerate(params):
                    pg = cur.pages[i]
                    ind = params[index]
                    pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
                    pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
                    pg.cropbox.lower_left = (ind['upper_left'][0], ind['upper_left'][1]-diff*20)
                    pg.cropbox.lower_right = (ind['upper_right'][0], ind['upper_right'][1]-diff*20)
                    output.add_page(pg)
    if not check_end:
        # proceed to standard crop
        for key, index in enumerate(params):
            pg = cur.pages[i]
            ind = params[index]
            pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
            pg.cropbox.lower_right = (ind['lower_right'][0], ind['lower_right'][1])
            pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
            pg.cropbox.lower_left = (ind['lower_left'][0], ind['lower_left'][1])
            output.add_page(pg)
            
print(counter + len(output.pages), len(cur.pages))

with open(f'./Created PDFs/{pdfs}.pdf', "wb") as out_f:
    output.write(out_f)

1819 625


In [125]:
for pdfs in os.listdir('./PDFS/'):
    cur = PDFReader(f'./PDFS/{pdfs}')
    output = PDFWriter()
    
    for i in range(len(cur.pages)):
        page_content = cur.pages[i].extract_text().split('\n')
        check_end, current_index = False, -1
        for ind in range(len(page_content)):
            if page_content[ind] == end_line:
                print(page_content[ind], i)
                check_end = True
                current_index = ind
                break
        if check_end:
            # handle case where the end of the list for a certain precinct is found
            first_number = page_content[8].split()[0]
            if first_number == 'Name':
                continue
            else:
                # handle the case where the eight index value is not a valid table entry (Name and Signature)
                # print(first_number, current_index)
                continue
        else:
            # proceed to standard crop
            break
            for key, index in enumerate(params):
                pg = cur.pages[i]
                ind = params[index]
                pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
                pg.cropbox.lower_right = (ind['lower_right'][0], ind['lower_right'][1])
                pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
                pg.cropbox.lower_left = (ind['lower_left'][0], ind['lower_left'][1])
                output.add_page(pg)
    with open(f'./Created PDFs/{pdfs}.pdf', "wb") as out_f:
        output.write(out_f)
    break